In [ ]:
import os
import re
import pandas as pd
from datetime import datetime
from matplotlib import pyplot as plt
import seaborn as sns

# Data Collection

In [ ]:
# List of csv files
files = ['data/19219343.csv', 'data/23234816.csv', 'data/31521347.csv']

# Use a list comprehension to read each file into a DataFrame and concatenate them
merged = pd.concat([pd.read_csv(f, sep=',') for f in files], ignore_index=True, sort=True)

## Future Plans: 

- Connect Bank API
- Apply a merkle tree that can be used to verify the integrity of the data

In [ ]:
# sort by date
merged['Date'] = pd.to_datetime(merged['Date'], format='%d/%m/%Y')
# split Memo into two strings BARCLAYS PINGIT       	FROM NINTENDO SWIC PN e.g. BARCLAYS PINGIT
merged["Name"] = merged["Memo"].str.split("\t").str[0]
merged["Description"] = merged["Memo"].str.split("\t").str[1]
merged = merged.sort_values(by=['Date'])

# rename number to ID
merged = merged.rename(columns={'Number': 'ID'})
merged = merged.reset_index(drop=True)

In [ ]:
merged.head()

In [ ]:
# types of data
merged.dtypes 

In [ ]:
# change ID to a string
merged['ID'] = merged['ID'].astype(str)

In [ ]:
# types of data
merged.dtypes 

In [ ]:
merged['ID'].value_counts()[1:] # 1: to remove the 0 as it its not associated with a transaction name. 

In [ ]:
target_id = merged['ID'].value_counts()[1:].index.tolist()
target_id

In [ ]:
# return a set of all names 
names = merged['Name'].unique()

# catorgorise the names into groups

list(names)


In [ ]:
# return only the Memo column for the target ID and amount associated with it
def get_memo(id):
    df = merged[merged['ID'] == id]
    df = df[['Memo', 'Amount']]
    df = df.reset_index(drop=True)
    return df

for i in target_id:
    print(i, get_memo(i))

This does Identify some recurring payments but not all of them. Looking at most common amounts might give us a clue about what is going on.

A recurring payment is a type of financial transaction where a customer provides authorization to a business or service provider to automatically charge their account at regular intervals for products or services. The conditions that classify a payment as recurring typically include:

1. **Regular Frequency:** Recurring payments occur at regular intervals, such as weekly, monthly, quarterly, or annually. The frequency is predetermined and agreed upon between the payer and the payee.

2. **Authorization:** The payer must provide explicit authorization for the recurring payment. This authorization may be given through various means, including signed agreements, online forms, or terms and conditions agreed to during the sign-up process.

3. **Fixed or Variable Amounts:** Recurring payments can involve fixed amounts (e.g., subscription fees) or variable amounts (e.g., utility bills based on usage). The key is that the payment is expected to occur regularly.

4. **Automated Processing:** Recurring payments are typically processed automatically without requiring manual intervention from the payer for each transaction. Automated systems handle the payment based on the agreed-upon schedule.

5. **Continuity of Service or Product:** Recurring payments are often associated with ongoing services or subscriptions. As long as the service or subscription continues, the recurring payments will persist.

Common examples of recurring payments include:

- **Subscription Services:** Monthly fees for streaming services, magazines, or software subscriptions.
- **Utility Bills:** Monthly bills for electricity, water, gas, etc.
- **Loan Repayments:** Regular payments toward a loan or mortgage.
- **Membership Fees:** Regular fees for gym memberships, clubs, or professional associations.

It's important for businesses to be transparent about their recurring payment policies and ensure that customers are aware of the terms and conditions before authorizing such transactions. Additionally, customers should have the ability to easily cancel or modify recurring payments when necessary.

In [ ]:
# add a column that indentifies the type of transaction expense or income
merged['Type'] = merged['Amount'].apply(lambda x: 'Expense' if x < 0 else 'Income')

In [ ]:
def check_recurring(id):
    """Identify recurring transactions based on the date and amount and ID. 
    
    Keyword arguments:
    argument -- description
    Return: return_description
    """
    pass 

In [ ]:
# add a day of the month column
merged['Day']= merged['Date'].dt.day
merged['M/Y'] = merged['Date'].dt.strftime('%m/%Y')

In [ ]:
# Example: Grouping by transaction amount and date
grouped_expense = merged[merged['Type'] == 'Expense'].groupby(['Day', 'Amount']).size().reset_index(name='TransactionCount')

# Example: Filter transactions with a count greater than a threshold
recurring_payments = grouped_expense[grouped_expense['TransactionCount'] > 3]


In [ ]:
recurring_payments.head()
# if recurring payments appears in merged dataframe, then it is a recurring payment tag it 

merged['Recurring'] = merged.apply(lambda x: True if x['Day'] in recurring_payments['Day'].values and x['Amount'] in recurring_payments['Amount'].values else False, axis=1)

In [ ]:
merged.head()

In [ ]:
def categorize_transaction(description):
    description = description.lower()

    # e.g grocery_keywords = ['tesco', 'asda', 'sainsbury']

    grocery_keywords = []
    dining_keywords = []
    online_shopping_keywords = []
    transportation_keywords = []
    utilities_keywords = []
    entertainment_keywords = []
    subscription_keywords = []
    clothing_keywords = []
    bank_fees_keywords = []
    financing_keywords = []
    savings_keywords = []
    trading_keywords = []

    for keyword_list, category in zip(
        [grocery_keywords, dining_keywords, online_shopping_keywords, transportation_keywords,
         utilities_keywords, entertainment_keywords, subscription_keywords, clothing_keywords,
         bank_fees_keywords, financing_keywords, savings_keywords, trading_keywords],
        ['Groceries', 'Dining Out', 'Online Shopping', 'Transportation', 'Utilities', 'Entertainment',
            'Subscription Services', 'Clothing and Accessories', 'Bank Fees', 'Financing', 'Savings', 'Trading']
    ):
        for keyword in keyword_list:
            if keyword in description:
                return category

    return 'Miscellaneous'


# Example Usage:
# transactions = ['MCDONALDS', 'AMAZON.CO.UK*V697Y LUXEMBOURG']
# for transaction in transactions:
#     category = categorize_transaction(transaction)
#     print(f'Transaction: {transaction}, Category: {category}')


merged['Category'] = merged['Name'].apply(categorize_transaction)



In [ ]:
# if subcategory is a funds transfer and the memo starts with a 6 digit number, then it is a transfer in the catergory column if it does not start with a 6 digit number then it is a payee transfer

merged['Category'] = merged.apply(lambda x: 'Transfer' if x['Subcategory'] == 'Funds Transfer' and re.match(r'^\d{6}', x['Memo']) else x['Category'], axis=1)

merged['Category'] = merged.apply(lambda x: 'Payee Transfer' if x['Subcategory'] == 'Funds Transfer' and not re.match(r'^\d{6}', x['Memo']) else x['Category'], axis=1)

# mark catergory as an income if the amount is greater than 0
merged['Category'] = merged.apply(lambda x: 'Income' if x['Amount'] > 0 else x['Category'], axis=1)

In [ ]:
# remove all transfers from the dataframe
merged = merged[merged['Category'] != 'Transfer']

In [ ]:
merged.head()

# Plotting

In [ ]:
# Example: Plotting a histogram of recurring payment counts

recurring_payments['TransactionCount'].hist()
plt.xlabel('Number of Recurring Payments')
plt.ylabel('Frequency')
plt.show()


In [ ]:
# plot the number of transactions per day
merged['Date'].value_counts().plot()

# Cleaning the Data

In [ ]:
# create a new dataframe around the category column with totals for each month 
# e.g. Groceries 100, Dining Out 200, Online Shopping 300, etc for each month

# Assuming df is your original DataFrame and it has columns 'category', 'month' and 'amount'
# 'month' is the column that represents the month of each transaction
# 'amount' is the column that represents the amount of each transaction but it must be negative for expenses 

grouped_merge = merged.groupby(['Category', 'M/Y'])['Amount'].sum().sort_values(ascending=False).reset_index(name='Total Amount')
# This will give you a new DataFrame with each row representing a unique category-month combination and the total amount for that combination.

# sort by m/y which is the month and year
# grouped_merge['M/Y'] = pd.to_datetime(grouped_merge['M/Y'], format='%m/%Y')
grouped_merge = grouped_merge.sort_values(by=['M/Y'])

In [ ]:
grouped_merge.head()

In [ ]:
grouped_merge.dtypes

# Output

In [ ]:
grouped_merge.to_pickle('data/merged.pkl')
grouped_merge.to_csv('data/groupm.csv', index=False)
merged.to_csv('data/merged.csv', index=False)

In [ ]:
# later on, well use a merkle tree to verify the data integrity and regularly check the data with new data from barclays api. 